In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from importlib import reload
import pandas as pd
import ipywidget_pivot_table as pt
from time import sleep

In [3]:
df = pt.samples.df_tips

dic = {
    'sumOverSum': '$.pivotUtilities.aggregators["Sum over Sum"]',
    'heatmap': '$.pivotUtilities.renderers["Heatmap"]'    
}
options = {
    'rows': ["sex", "smoker"],
    'cols': ["day", "time"],
    'vals': ["tip", "total_bill"],
    'aggregator': '{sumOverSum}(["tip", "total_bill"])'.format(**dic),
    'renderer': '{heatmap}'.format(**dic)
}

p1 = pt.Pivot(df_data=df, options=options)
p1

Pivot(data=[['row', 'total_bill', 'tip', 'sex', 'smoker', 'day', 'time', 'size'], [1, 16.99, 1.01, 'Female', 'No', 'Sun', 'Dinner', 2], [2, 10.34, 1.66, 'Male', 'No', 'Sun', 'Dinner', 3], [3, 21.01, 3.5, 'Male', 'No', 'Sun', 'Dinner', 3], [4, 23.68, 3.31, 'Male', 'No', 'Sun', 'Dinner', 2], [5, 24.59, 3.61, 'Female', 'No', 'Sun', 'Dinner', 4], [6, 25.29, 4.71, 'Male', 'No', 'Sun', 'Dinner', 4], [7, 8.77, 2.0, 'Male', 'No', 'Sun', 'Dinner', 2], [8, 26.88, 3.12, 'Male', 'No', 'Sun', 'Dinner', 4], [9, 15.04, 1.96, 'Male', 'No', 'Sun', 'Dinner', 2], [10, 14.78, 3.23, 'Male', 'No', 'Sun', 'Dinner', 2], [11, 10.27, 1.71, 'Male', 'No', 'Sun', 'Dinner', 2], [12, 35.26, 5.0, 'Female', 'No', 'Sun', 'Dinner', 4], [13, 15.42, 1.57, 'Male', 'No', 'Sun', 'Dinner', 2], [14, 18.43, 3.0, 'Male', 'No', 'Sun', 'Dinner', 4], [15, 14.83, 3.02, 'Female', 'No', 'Sun', 'Dinner', 2], [16, 21.58, 3.92, 'Male', 'No', 'Sun', 'Dinner', 2], [17, 10.33, 1.67, 'Female', 'No', 'Sun', 'Dinner', 3], [18, 16.29, 3.71, 'Male', 'No', 'Sun', 'Dinner', 3], [19, 16.97, 3.5, 'Female', 'No', 'Sun', 'Dinner', 3], [20, 20.65, 3.35, 'Male', 'No', 'Sat', 'Dinner', 3], [21, 17.92, 4.08, 'Male', 'No', 'Sat', 'Dinner', 2], [22, 20.29, 2.75, 'Female', 'No', 'Sat', 'Dinner', 2], [23, 15.77, 2.23, 'Female', 'No', 'Sat', 'Dinner', 2], [24, 39.42, 7.58, 'Male', 'No', 'Sat', 'Dinner', 4], [25, 19.82, 3.18, 'Male', 'No', 'Sat', 'Dinner', 2], [26, 17.81, 2.34, 'Male', 'No', 'Sat', 'Dinner', 4], [27, 13.37, 2.0, 'Male', 'No', 'Sat', 'Dinner', 2], [28, 12.69, 2.0, 'Male', 'No', 'Sat', 'Dinner', 2], [29, 21.7, 4.3, 'Male', 'No', 'Sat', 'Dinner', 2], [30, 19.65, 3.0, 'Female', 'No', 'Sat', 'Dinner', 2], [31, 9.55, 1.45, 'Male', 'No', 'Sat', 'Dinner', 2], [32, 18.35, 2.5, 'Male', 'No', 'Sat', 'Dinner', 4], [33, 15.06, 3.0, 'Female', 'No', 'Sat', 'Dinner', 2], [34, 20.69, 2.45, 'Female', 'No', 'Sat', 'Dinner', 4], [35, 17.78, 3.27, 'Male', 'No', 'Sat', 'Dinner', 2], [36, 24.06, 3.6, 'Male', 'No', 'Sat', 'Dinner', 3], [37, 16.31, 2.0, 'Male', 'No', 'Sat', 'Dinner', 3], [38, 16.93, 3.07, 'Female', 'No', 'Sat', 'Dinner', 3], [39, 18.69, 2.31, 'Male', 'No', 'Sat', 'Dinner', 3], [40, 31.27, 5.0, 'Male', 'No', 'Sat', 'Dinner', 3], [41, 16.04, 2.24, 'Male', 'No', 'Sat', 'Dinner', 3], [42, 17.46, 2.54, 'Male', 'No', 'Sun', 'Dinner', 2], [43, 13.94, 3.06, 'Male', 'No', 'Sun', 'Dinner', 2], [44, 9.68, 1.32, 'Male', 'No', 'Sun', 'Dinner', 2], [45, 30.4, 5.6, 'Male', 'No', 'Sun', 'Dinner', 4], [46, 18.29, 3.0, 'Male', 'No', 'Sun', 'Dinner', 2], [47, 22.23, 5.0, 'Male', 'No', 'Sun', 'Dinner', 2], [48, 32.4, 6.0, 'Male', 'No', 'Sun', 'Dinner', 4], [49, 28.55, 2.05, 'Male', 'No', 'Sun', 'Dinner', 3], [50, 18.04, 3.0, 'Male', 'No', 'Sun', 'Dinner', 2], [51, 12.54, 2.5, 'Male', 'No', 'Sun', 'Dinner', 2], [52, 10.29, 2.6, 'Female', 'No', 'Sun', 'Dinner', 2], [53, 34.81, 5.2, 'Female', 'No', 'Sun', 'Dinner', 4], [54, 9.94, 1.56, 'Male', 'No', 'Sun', 'Dinner', 2], [55, 25.56, 4.34, 'Male', 'No', 'Sun', 'Dinner', 4], [56, 19.49, 3.51, 'Male', 'No', 'Sun', 'Dinner', 2], [57, 38.01, 3.0, 'Male', 'Yes', 'Sat', 'Dinner', 4], [58, 26.41, 1.5, 'Female', 'No', 'Sat', 'Dinner', 2], [59, 11.24, 1.76, 'Male', 'Yes', 'Sat', 'Dinner', 2], [60, 48.27, 6.73, 'Male', 'No', 'Sat', 'Dinner', 4], [61, 20.29, 3.21, 'Male', 'Yes', 'Sat', 'Dinner', 2], [62, 13.81, 2.0, 'Male', 'Yes', 'Sat', 'Dinner', 2], [63, 11.02, 1.98, 'Male', 'Yes', 'Sat', 'Dinner', 2], [64, 18.29, 3.76, 'Male', 'Yes', 'Sat', 'Dinner', 4], [65, 17.59, 2.64, 'Male', 'No', 'Sat', 'Dinner', 3], [66, 20.08, 3.15, 'Male', 'No', 'Sat', 'Dinner', 3], [67, 16.45, 2.47, 'Female', 'No', 'Sat', 'Dinner', 2], [68, 3.07, 1.0, 'Female', 'Yes', 'Sat', 'Dinner', 1], [69, 20.23, 2.01, 'Male', 'No', 'Sat', 'Dinner', 2], [70, 15.01, 2.09, 'Male', 'Yes', 'Sat', 'Dinner', 2], [71, 12.02, 1.97, 'Male', 'No', 'Sat', 'Dinner', 2], [72, 17.07, 3.0, 'Female', 'No', 'Sat', 'Dinner', 3], [73, 26.86, 3.14, 'Female', 'Yes', 'Sat', 'Dinner', 2], [74, 25.28, 5.0, 'Female', 'Yes', 'Sat', 'Dinn

In [4]:
p1.options = {
    'rows': ["smoker", "sex"],
    'cols': ["day", "time"],
    'vals': ["tip", "total_bill"],
    'aggregator': '{sumOverSum}(["tip", "total_bill"])'.format(**dic),
    'renderer': '$.pivotUtilities.renderers["Table"]'
}

In [5]:
p1.options_init

{'aggregator': '$.pivotUtilities.aggregators["Sum over Sum"](["tip", "total_bill"])',
 'cols': ['day', 'time'],
 'renderer': '$.pivotUtilities.renderers["Heatmap"]',
 'rows': ['sex', 'smoker'],
 'vals': ['tip', 'total_bill']}

In [6]:
df = pt.samples.df_tips

dic = {
    'sumOverSum': '$.pivotUtilities.aggregators["Sum over Sum"]',
    'heatmap': '$.pivotUtilities.renderers["Heatmap"]'    
}
options = {
    'rows': ["sex", "smoker"],
    'cols': ["day", "time"],
    'vals': ["tip", "total_bill"],
    'aggregatorName': 'Sum over Sum',
    'rendererName': 'Heatmap'
}

p2 = pt.PivotUI(df_data=df, options=options)
p2

PivotUI(data=[['row', 'total_bill', 'tip', 'sex', 'smoker', 'day', 'time', 'size'], [1, 16.99, 1.01, 'Female', 'No', 'Sun', 'Dinner', 2], [2, 10.34, 1.66, 'Male', 'No', 'Sun', 'Dinner', 3], [3, 21.01, 3.5, 'Male', 'No', 'Sun', 'Dinner', 3], [4, 23.68, 3.31, 'Male', 'No', 'Sun', 'Dinner', 2], [5, 24.59, 3.61, 'Female', 'No', 'Sun', 'Dinner', 4], [6, 25.29, 4.71, 'Male', 'No', 'Sun', 'Dinner', 4], [7, 8.77, 2.0, 'Male', 'No', 'Sun', 'Dinner', 2], [8, 26.88, 3.12, 'Male', 'No', 'Sun', 'Dinner', 4], [9, 15.04, 1.96, 'Male', 'No', 'Sun', 'Dinner', 2], [10, 14.78, 3.23, 'Male', 'No', 'Sun', 'Dinner', 2], [11, 10.27, 1.71, 'Male', 'No', 'Sun', 'Dinner', 2], [12, 35.26, 5.0, 'Female', 'No', 'Sun', 'Dinner', 4], [13, 15.42, 1.57, 'Male', 'No', 'Sun', 'Dinner', 2], [14, 18.43, 3.0, 'Male', 'No', 'Sun', 'Dinner', 4], [15, 14.83, 3.02, 'Female', 'No', 'Sun', 'Dinner', 2], [16, 21.58, 3.92, 'Male', 'No', 'Sun', 'Dinner', 2], [17, 10.33, 1.67, 'Female', 'No', 'Sun', 'Dinner', 3], [18, 16.29, 3.71, 'Male', 'No', 'Sun', 'Dinner', 3], [19, 16.97, 3.5, 'Female', 'No', 'Sun', 'Dinner', 3], [20, 20.65, 3.35, 'Male', 'No', 'Sat', 'Dinner', 3], [21, 17.92, 4.08, 'Male', 'No', 'Sat', 'Dinner', 2], [22, 20.29, 2.75, 'Female', 'No', 'Sat', 'Dinner', 2], [23, 15.77, 2.23, 'Female', 'No', 'Sat', 'Dinner', 2], [24, 39.42, 7.58, 'Male', 'No', 'Sat', 'Dinner', 4], [25, 19.82, 3.18, 'Male', 'No', 'Sat', 'Dinner', 2], [26, 17.81, 2.34, 'Male', 'No', 'Sat', 'Dinner', 4], [27, 13.37, 2.0, 'Male', 'No', 'Sat', 'Dinner', 2], [28, 12.69, 2.0, 'Male', 'No', 'Sat', 'Dinner', 2], [29, 21.7, 4.3, 'Male', 'No', 'Sat', 'Dinner', 2], [30, 19.65, 3.0, 'Female', 'No', 'Sat', 'Dinner', 2], [31, 9.55, 1.45, 'Male', 'No', 'Sat', 'Dinner', 2], [32, 18.35, 2.5, 'Male', 'No', 'Sat', 'Dinner', 4], [33, 15.06, 3.0, 'Female', 'No', 'Sat', 'Dinner', 2], [34, 20.69, 2.45, 'Female', 'No', 'Sat', 'Dinner', 4], [35, 17.78, 3.27, 'Male', 'No', 'Sat', 'Dinner', 2], [36, 24.06, 3.6, 'Male', 'No', 'Sat', 'Dinner', 3], [37, 16.31, 2.0, 'Male', 'No', 'Sat', 'Dinner', 3], [38, 16.93, 3.07, 'Female', 'No', 'Sat', 'Dinner', 3], [39, 18.69, 2.31, 'Male', 'No', 'Sat', 'Dinner', 3], [40, 31.27, 5.0, 'Male', 'No', 'Sat', 'Dinner', 3], [41, 16.04, 2.24, 'Male', 'No', 'Sat', 'Dinner', 3], [42, 17.46, 2.54, 'Male', 'No', 'Sun', 'Dinner', 2], [43, 13.94, 3.06, 'Male', 'No', 'Sun', 'Dinner', 2], [44, 9.68, 1.32, 'Male', 'No', 'Sun', 'Dinner', 2], [45, 30.4, 5.6, 'Male', 'No', 'Sun', 'Dinner', 4], [46, 18.29, 3.0, 'Male', 'No', 'Sun', 'Dinner', 2], [47, 22.23, 5.0, 'Male', 'No', 'Sun', 'Dinner', 2], [48, 32.4, 6.0, 'Male', 'No', 'Sun', 'Dinner', 4], [49, 28.55, 2.05, 'Male', 'No', 'Sun', 'Dinner', 3], [50, 18.04, 3.0, 'Male', 'No', 'Sun', 'Dinner', 2], [51, 12.54, 2.5, 'Male', 'No', 'Sun', 'Dinner', 2], [52, 10.29, 2.6, 'Female', 'No', 'Sun', 'Dinner', 2], [53, 34.81, 5.2, 'Female', 'No', 'Sun', 'Dinner', 4], [54, 9.94, 1.56, 'Male', 'No', 'Sun', 'Dinner', 2], [55, 25.56, 4.34, 'Male', 'No', 'Sun', 'Dinner', 4], [56, 19.49, 3.51, 'Male', 'No', 'Sun', 'Dinner', 2], [57, 38.01, 3.0, 'Male', 'Yes', 'Sat', 'Dinner', 4], [58, 26.41, 1.5, 'Female', 'No', 'Sat', 'Dinner', 2], [59, 11.24, 1.76, 'Male', 'Yes', 'Sat', 'Dinner', 2], [60, 48.27, 6.73, 'Male', 'No', 'Sat', 'Dinner', 4], [61, 20.29, 3.21, 'Male', 'Yes', 'Sat', 'Dinner', 2], [62, 13.81, 2.0, 'Male', 'Yes', 'Sat', 'Dinner', 2], [63, 11.02, 1.98, 'Male', 'Yes', 'Sat', 'Dinner', 2], [64, 18.29, 3.76, 'Male', 'Yes', 'Sat', 'Dinner', 4], [65, 17.59, 2.64, 'Male', 'No', 'Sat', 'Dinner', 3], [66, 20.08, 3.15, 'Male', 'No', 'Sat', 'Dinner', 3], [67, 16.45, 2.47, 'Female', 'No', 'Sat', 'Dinner', 2], [68, 3.07, 1.0, 'Female', 'Yes', 'Sat', 'Dinner', 1], [69, 20.23, 2.01, 'Male', 'No', 'Sat', 'Dinner', 2], [70, 15.01, 2.09, 'Male', 'Yes', 'Sat', 'Dinner', 2], [71, 12.02, 1.97, 'Male', 'No', 'Sat', 'Dinner', 2], [72, 17.07, 3.0, 'Female', 'No', 'Sat', 'Dinner', 3], [73, 26.86, 3.14, 'Female', 'Yes', 'Sat', 'Dinner', 2], [74, 25.28, 5.0, 'Female', 'Yes', 'Sat', 'Di

In [7]:
p2.options_init

{'aggregatorName': 'Sum over Sum',
 'cols': ['day', 'time'],
 'rendererName': 'Heatmap',
 'rows': ['sex', 'smoker'],
 'vals': ['tip', 'total_bill']}

In [8]:
p2.options

{'aggregatorName': 'Sum over Sum',
 'cols': ['day', 'time'],
 'rendererName': 'Heatmap',
 'rows': ['sex', 'smoker'],
 'vals': ['tip', 'total_bill']}

In [9]:
p2

PivotUI(data=[['row', 'total_bill', 'tip', 'sex', 'smoker', 'day', 'time', 'size'], [1, 16.99, 1.01, 'Female', 'No', 'Sun', 'Dinner', 2], [2, 10.34, 1.66, 'Male', 'No', 'Sun', 'Dinner', 3], [3, 21.01, 3.5, 'Male', 'No', 'Sun', 'Dinner', 3], [4, 23.68, 3.31, 'Male', 'No', 'Sun', 'Dinner', 2], [5, 24.59, 3.61, 'Female', 'No', 'Sun', 'Dinner', 4], [6, 25.29, 4.71, 'Male', 'No', 'Sun', 'Dinner', 4], [7, 8.77, 2.0, 'Male', 'No', 'Sun', 'Dinner', 2], [8, 26.88, 3.12, 'Male', 'No', 'Sun', 'Dinner', 4], [9, 15.04, 1.96, 'Male', 'No', 'Sun', 'Dinner', 2], [10, 14.78, 3.23, 'Male', 'No', 'Sun', 'Dinner', 2], [11, 10.27, 1.71, 'Male', 'No', 'Sun', 'Dinner', 2], [12, 35.26, 5.0, 'Female', 'No', 'Sun', 'Dinner', 4], [13, 15.42, 1.57, 'Male', 'No', 'Sun', 'Dinner', 2], [14, 18.43, 3.0, 'Male', 'No', 'Sun', 'Dinner', 4], [15, 14.83, 3.02, 'Female', 'No', 'Sun', 'Dinner', 2], [16, 21.58, 3.92, 'Male', 'No', 'Sun', 'Dinner', 2], [17, 10.33, 1.67, 'Female', 'No', 'Sun', 'Dinner', 3], [18, 16.29, 3.71, 'Male', 'No', 'Sun', 'Dinner', 3], [19, 16.97, 3.5, 'Female', 'No', 'Sun', 'Dinner', 3], [20, 20.65, 3.35, 'Male', 'No', 'Sat', 'Dinner', 3], [21, 17.92, 4.08, 'Male', 'No', 'Sat', 'Dinner', 2], [22, 20.29, 2.75, 'Female', 'No', 'Sat', 'Dinner', 2], [23, 15.77, 2.23, 'Female', 'No', 'Sat', 'Dinner', 2], [24, 39.42, 7.58, 'Male', 'No', 'Sat', 'Dinner', 4], [25, 19.82, 3.18, 'Male', 'No', 'Sat', 'Dinner', 2], [26, 17.81, 2.34, 'Male', 'No', 'Sat', 'Dinner', 4], [27, 13.37, 2.0, 'Male', 'No', 'Sat', 'Dinner', 2], [28, 12.69, 2.0, 'Male', 'No', 'Sat', 'Dinner', 2], [29, 21.7, 4.3, 'Male', 'No', 'Sat', 'Dinner', 2], [30, 19.65, 3.0, 'Female', 'No', 'Sat', 'Dinner', 2], [31, 9.55, 1.45, 'Male', 'No', 'Sat', 'Dinner', 2], [32, 18.35, 2.5, 'Male', 'No', 'Sat', 'Dinner', 4], [33, 15.06, 3.0, 'Female', 'No', 'Sat', 'Dinner', 2], [34, 20.69, 2.45, 'Female', 'No', 'Sat', 'Dinner', 4], [35, 17.78, 3.27, 'Male', 'No', 'Sat', 'Dinner', 2], [36, 24.06, 3.6, 'Male', 'No', 'Sat', 'Dinner', 3], [37, 16.31, 2.0, 'Male', 'No', 'Sat', 'Dinner', 3], [38, 16.93, 3.07, 'Female', 'No', 'Sat', 'Dinner', 3], [39, 18.69, 2.31, 'Male', 'No', 'Sat', 'Dinner', 3], [40, 31.27, 5.0, 'Male', 'No', 'Sat', 'Dinner', 3], [41, 16.04, 2.24, 'Male', 'No', 'Sat', 'Dinner', 3], [42, 17.46, 2.54, 'Male', 'No', 'Sun', 'Dinner', 2], [43, 13.94, 3.06, 'Male', 'No', 'Sun', 'Dinner', 2], [44, 9.68, 1.32, 'Male', 'No', 'Sun', 'Dinner', 2], [45, 30.4, 5.6, 'Male', 'No', 'Sun', 'Dinner', 4], [46, 18.29, 3.0, 'Male', 'No', 'Sun', 'Dinner', 2], [47, 22.23, 5.0, 'Male', 'No', 'Sun', 'Dinner', 2], [48, 32.4, 6.0, 'Male', 'No', 'Sun', 'Dinner', 4], [49, 28.55, 2.05, 'Male', 'No', 'Sun', 'Dinner', 3], [50, 18.04, 3.0, 'Male', 'No', 'Sun', 'Dinner', 2], [51, 12.54, 2.5, 'Male', 'No', 'Sun', 'Dinner', 2], [52, 10.29, 2.6, 'Female', 'No', 'Sun', 'Dinner', 2], [53, 34.81, 5.2, 'Female', 'No', 'Sun', 'Dinner', 4], [54, 9.94, 1.56, 'Male', 'No', 'Sun', 'Dinner', 2], [55, 25.56, 4.34, 'Male', 'No', 'Sun', 'Dinner', 4], [56, 19.49, 3.51, 'Male', 'No', 'Sun', 'Dinner', 2], [57, 38.01, 3.0, 'Male', 'Yes', 'Sat', 'Dinner', 4], [58, 26.41, 1.5, 'Female', 'No', 'Sat', 'Dinner', 2], [59, 11.24, 1.76, 'Male', 'Yes', 'Sat', 'Dinner', 2], [60, 48.27, 6.73, 'Male', 'No', 'Sat', 'Dinner', 4], [61, 20.29, 3.21, 'Male', 'Yes', 'Sat', 'Dinner', 2], [62, 13.81, 2.0, 'Male', 'Yes', 'Sat', 'Dinner', 2], [63, 11.02, 1.98, 'Male', 'Yes', 'Sat', 'Dinner', 2], [64, 18.29, 3.76, 'Male', 'Yes', 'Sat', 'Dinner', 4], [65, 17.59, 2.64, 'Male', 'No', 'Sat', 'Dinner', 3], [66, 20.08, 3.15, 'Male', 'No', 'Sat', 'Dinner', 3], [67, 16.45, 2.47, 'Female', 'No', 'Sat', 'Dinner', 2], [68, 3.07, 1.0, 'Female', 'Yes', 'Sat', 'Dinner', 1], [69, 20.23, 2.01, 'Male', 'No', 'Sat', 'Dinner', 2], [70, 15.01, 2.09, 'Male', 'Yes', 'Sat', 'Dinner', 2], [71, 12.02, 1.97, 'Male', 'No', 'Sat', 'Dinner', 2], [72, 17.07, 3.0, 'Female', 'No', 'Sat', 'Dinner', 3], [73, 26.86, 3.14, 'Female', 'Yes', 'Sat', 'Dinner', 2], [74, 25.28, 5.0, 'Female', 'Yes', 'Sat', 'Di